In [9]:
# !pip install xgboost

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 297.1 MB 357 kB/s eta 0:00:01     |███████████████████████████▎    | 253.5 MB 386 kB/s eta 0:01:53     |███████████████████████████▍    | 254.1 MB 290 kB/s eta 0:02:28     |████████████████████████████▉   | 267.7 MB 337 kB/s eta 0:01:28


In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

# 加载数据
data = pd.read_csv('./combined_data.csv')

# 删除不需要的列，例如时间戳或IP地址（假设你的数据集中有这些列）
data.drop([' Timestamp'], axis=1, inplace=True)

# 类型转换，将分类标签编码
label_encoder = LabelEncoder()
data[' Label'] = label_encoder.fit_transform(data[' Label'])

# 检查并处理无穷大和非常大的数值
data.replace([np.inf, -np.inf], np.nan, inplace=True)  # 将inf替换为NaN
data.fillna(data.median(), inplace=True)  # 使用中位数填充NaN，确保之前中位数计算不包括inf

# 特征标准化
scaler = StandardScaler()
X = scaler.fit_transform(data.drop(' Label', axis=1))  # 确保标签列不参与标准化
y = data[' Label']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("数据预处理完成，准备进行模型训练和测试。")


数据预处理完成，准备进行模型训练和测试。


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# 初始化模型
logreg = LogisticRegression(max_iter=1000)
rf = RandomForestClassifier(n_estimators=100)
svm = SVC()
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# 训练逻辑回归模型
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)
print("Logistic Regression Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred_logreg) * 100))

# 训练随机森林模型
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Random Forest Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred_rf) * 100))

# 训练支持向量机模型
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print("SVM Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred_svm) * 100))

# 训练XGBoost模型
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
print("XGBoost Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred_xgb) * 100))

# 打印分类报告（以XGBoost为例）
print("\nClassification Report for XGBoost:")
print(classification_report(y_test, y_pred_xgb))


Logistic Regression Accuracy: 54.96%
Random Forest Accuracy: 62.04%
SVM Accuracy: 50.17%
XGBoost Accuracy: 62.75%

Classification Report for XGBoost:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       170
           1       0.50      0.42      0.45       143
           2       0.31      0.25      0.28       174
           3       0.56      0.52      0.54       159
           4       0.99      0.99      0.99       145
           5       0.45      0.42      0.43       146
           6       0.60      0.65      0.63       148
           7       0.46      0.55      0.50       121
           8       0.36      0.46      0.40       144
           9       0.54      0.56      0.55       156
          10       0.38      0.40      0.39       154
          11       0.40      0.44      0.42       146
          12       0.99      0.98      0.99       150
          13       1.00      0.97      0.99       158
          14       0.51      0.49      